In [ ]:

from utils.utils import Utils as ut
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LambdaCallback
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import EfficientNetB0, DenseNet121, MobileNetV2, ResNet50
from sklearn.metrics import confusion_matrix, classification_report
from keras.metrics import AUC
from keras.layers import Dropout, Dense, GlobalAveragePooling2D, Flatten, LeakyReLU, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from keras.initializers import GlorotUniform
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from keras.utils import to_categorical
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow import keras
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import cv2
import os
import io

In [ ]:
labels = ['glioma', 'meningioma', 'no_tumor', 'pituitary']
testing_dir = 'Testing/'
training_dir = 'Training/'

In [ ]:
subfolders = ut.get_subfolders(training_dir)
ut.display_images_from_subfolders(training_dir, subfolders)

In [ ]:
X_train, y_train = ut.load_data(training_dir, labels)
X_test, y_test = ut.load_data(testing_dir, labels)

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

y_train = to_categorical(y_train) # One-hot encoding of integer labels
y_test = to_categorical(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 20% of the data is validation data

In [ ]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

image_gen.fit(X_train)

In [ ]:
models = {
    'EfficientNet': ut.build_and_compile_model('EfficientNet', 224),
    'DenseNet': ut.build_and_compile_model('DenseNet', 224),
    'MobileNet': ut.build_and_compile_model('MobileNet', 224),
    'ResNet50': ut.build_and_compile_model('ResNet50', 224)
}

ut.display_summaries(models)

In [ ]:
history_dict = ut.train_models(models)

In [ ]:
for model_name, history in history_dict.items():
    ut.plot('loss', history)
    ut.plot('accuracy', history)
    ut.plot('auc', history)

In [ ]:
for model_name, model in models.items():
    predicted_classes = np.argmax(model.predict(X_test), axis=1)
    confusionmatrix = confusion_matrix(np.argmax(y_test,axis=1), predicted_classes)
    plt.figure(figsize = (16, 16))
    sns.heatmap(confusionmatrix, cmap='Blues', annot=True, cbar=True)

    print(classification_report(np.argmax(y_test,axis=1), predicted_classes))

In [ ]:
metrics_list = []

for model_name, model in models.items():
    loss, acc, auc = model.evaluate(X_test, y_test)
    metrics_list.append([model_name, f"{acc * 100:.2f}%", f"{auc * 100:.2f}%", f"{loss:.2f}"])

df = pd.DataFrame(metrics_list, columns=["Model", "Accuracy (%)", "AUC (%)", "Loss"])

print(df)

In [ ]:
model = Sequential([
    # First Conv Block
    Conv2D(32, (3, 3), kernel_initializer='he_normal', input_shape=(224, 224, 3)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),
    
    # Second Conv Block
    Conv2D(64, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),
    
    # Third Conv Block
    Conv2D(128, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),
    
    # New Fourth Conv Block
    Conv2D(256, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),

    # New Fifth Conv Block
    Conv2D(512, (3, 3), kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    MaxPooling2D((2, 2)),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(128, kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),
    
    Dense(64, kernel_initializer='he_normal'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),

    # Output Layer
    Dense(4, activation='softmax', kernel_regularizer=l2(0.01))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])
model.summary()


In [ ]:
model = ut.train_model('bt_cnn', image_gen, X_train, y_train, X_val, y_val, model)